In [1]:
from math import sqrt,sin,cos
from numpy import array
from numpy import linspace,zeros,float32
import numpy as np

In [2]:
import torch
if torch.cuda.is_available():
    use_gpu = True
    print("Using GPU")
else:
    use_gpu = False
FloatTensor = torch.cuda.FloatTensor if use_gpu else torch.FloatTensor
DoubleTensor = torch.cuda.DoubleTensor if use_gpu else torch.DoubleTensor
LongTensor = torch.cuda.LongTensor if use_gpu else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_gpu else torch.ByteTensor
Tensor = DoubleTensor

Using GPU


In [3]:
class theta:
    def __init__(
        self,
        count_state = 16,
        count_action = 5,
    ):
        self.count_state = count_state
        self.count_action = count_action
        self.matrix = np.zeros((count_action,count_state),dtype = np.float32)
            
    def __call__(self): return self.matrix
    
    def calc(
        self,
        s,
        a,
    ):
        return np.matmul(a.transpose(),np.matmul(self.matrix,s))
    
    def add_state(
        self,
    ):
        self.matrix = np.c_[self.matrix, np.zeros(self.count_action)]
        self.count_state += 1
        
    def add_action(
        self,
    ):
        self.matrix = np.r_[self.matrix, [np.zeros(self.count_state)]]
        self.count_action += 1

In [4]:
class dots:
    def __init__(
        self,
        array
    ):
        self.dim = len(array[0])
        self.count = len(array[:,0])
        self.array = array
    def __call__(
        self,
    ): return self.array
    def add(
        self,
        vec
    ):
        self.array = np.vstack((self(), vec))
        self.count += 1

In [124]:
class func:
    def __init__(
        self,
        basis_dots,
        kind = 'frobenius_norm',
    ):
        self.basis = basis_dots
        self.dim = basis_dots.dim
        self.count = basis_dots.count
        self.base_f = self.frobenius
        
    def __call__(self,): return self.frobenius_result()
    
    def add(
        self,
        vec
    ):
        self.basis.add(vec)
        self.count += 1
    
    def frobenius(
        self,
        dot,
    ):
        return lambda x: np.power(np.divide(1.,
            np.linalg.norm(x-dot),
            out = np.ones_like([1.])*np.inf,
            where = np.linalg.norm(x-dot)>self.dim/self.count
            ),4)
    
    def frobenius_result(
        self,
    ):       
        local_vect = lambda x: np.fromiter((self.base_f(b_v)(x) for b_v in self.basis())
             ,self.basis().dtype)
        norm = lambda x: np.sum(local_vect(x)) 
        normalized_local_vect = lambda x: np.divide(local_vect(x),
            norm(x),
            out = np.ones_like(local_vect(x)) / np.count_nonzero(local_vect(x) == np.inf),
            where = local_vect(x)!=np.inf
            )
        return lambda x: normalized_local_vect(x) 

In [125]:
class replay_buffer:
    def __init__(
        self,
        state_dim = 16,
        action_dim = 3,
        round_len = 50,
    ):
        [x for x in range(round_len)]
        self.value = dict.fromkeys([x for x in range(round_len)], 
                {"state":np.zeros((state_dim),dtype = float32),
                   "action":np.zeros((action_dim),dtype = float32),
                   "state_new":np.zeros((state_dim),dtype = float32),
                   "reward":0.0})
#         self.value = [{"t":x,
#                    "state":np.zeros((state_dim),dtype = float64),
#                    "action":np.zeros((action_dim),dtype = float64),
#                    "state_new":np.zeros((state_dim),dtype = float64),
#                    "reward":0.0} for x in range(round_len)]
        
    def __call__(self,): return self.value
    
    def state(self, t): return self.value[t]["state"]
    def action(self, t): return self.value[t]["action"]
    def reward(self, t): return self.value[t]["reward"]
    
    def replay_buffer_import(self,): return None    
    def replay_buffer_export(self,): return None
    

In [126]:
import random
import functools as f
import time

class agent:
    def __init__(
        self,
        state_basis,
        action_basis,
        state_func_type = 'frobenius_norm',
        action_func_type = 'frobenius_norm',
        action_scale = sqrt(2.0),
        gamma = 0.9,
        greedy_rate = 0.1,
        epsilon = 0.3,
        replay_buffer_size = 50,
        device = torch.device("cuda:0"),
    ):
        self.s_f = func(state_basis, state_func_type)
        self.a_f = func(action_basis, action_func_type)
        self.Q = theta(state_basis.count, action_basis.count)
        self.count_use = theta(state_basis.count, action_basis.count)
        self.count_use.matrix += 1.
        self.epoch_learn = 0
        self.round_learn = 0
        self.state = np.array([0.2, 0.3, 0.01, -0.05])
        self.state_prev = None
        self.action = None
        self.reward = 0.
        self.action_scale = action_scale
        self.replay_buffer = replay_buffer(state_basis.dim, action_basis.dim, replay_buffer_size)
        self.gamma = gamma
        self.greedy_rate = greedy_rate
        self.epsilon = epsilon
        self.replay_buffer_size = replay_buffer_size
        self.device = device
    
    def add_state(self):
        self.s_f.add()
        self.Q.add_state()
        self.count_use.add_state()
        self.count_use.matrix = np.where(count_use.matrix > 0.,count_use.matrix,1.)
        
    def add_action(self):
        self.a_f.add()
        self.Q.add_action()
        self.count_use.add_action()
        self.count_use.matrix = np.where(count_use.matrix > 0.,count_use.matrix,1.)
        
    def observe(self, state, reward):
        self.state = state
        self.reward = reward
    
    def policy(
        self,
        param = {"type":"random"},
    ):
        if param["type"] == "random":
            return (np.random.rand(self.a_f.dim) - 0.5) * 2.0 * self.action_scale
        elif param["type"] == "pd":
            k = np.array([[-0.12, 0, -0.68, 0],
                      [0, -0.12, 0, -0.68]])
            return np.dot(k, self.state )
        elif param["type"] == "greedy":
            greedy = random.choices(population=[True, False],
                    weights=[self.greedy_rate, 1.0 - self.greedy_rate],
                    k = 1)
            if greedy[0]:
                q_a_arr = np.fromiter(Q.calc(self.state, a) for a in self.a_f.basis())
                action_max_arr = np.argwhere(q_a_arr == np.min(q_a_arr))
                action_max_choice = random.choices(population = action_max_arr)
                return self.a_f.basis()[action_max_choice]
            else:
                return self.policy()
        else:
            return None
        
    #@f.lru_cache(maxsize=None)
    def Q_update(
        self,
    ):
#         p_1 = f.lru_cache(maxsize=None)(lambda t: np.outer(self.a_f()(self.replay_buffer()[t]["action"]),
#                                  self.s_f()(self.replay_buffer()[t]["state"])
#                                 ))
#         p_1 = f.lru_cache(maxsize=None)(lambda t: torch.from_numpy(np.outer(self.a_f()(self.replay_buffer.action(t)),
#                                  self.s_f()(self.replay_buffer.state(t))
#                                 )).float().to(self.device))
        
#         p_2 = (1.0 + self.s_f.dim * self.a_f.dim)/(torch.from_numpy(self.count_use.matrix).float().to(self.device) + self.s_f.dim * self.a_f.dim)
        
#         filter_by_epsilon = f.lru_cache(maxsize=None)(lambda t: torch.from_numpy(np.where(p_1(t).detach().cpu().numpy() > self.epsilon,
#                                                                                           1.0,0.0)).float().to(self.device))
        
#         p = f.lru_cache(maxsize=None)(lambda t: torch.multiply(filter_by_epsilon(t),
#                                   torch.multiply(p_1(t), p_2)))
        
#         T = f.lru_cache(maxsize=None)(lambda t: self.replay_buffer.reward(t) + self.gamma * np.min(np.matmul(self.Q.matrix,self.s_f()(self.replay_buffer.state(t)))))

#         #ticks_num = sum([filter_by_epsilon(t) for t in range(self.replay_buffer_size)])
#         time1 = time.time()
#         ticks_num = f.reduce(lambda x, y: x + filter_by_epsilon(y), range(self.replay_buffer_size)) 
#         time2 = time.time()
#         print('time diff ticks_num', time2-time1)
#         ticks_num_divided = torch.from_numpy(np.divide(1.0,
#                          ticks_num.detach().cpu().numpy(),
#                          out = np.zeros_like(ticks_num.detach().cpu().numpy()),
#                          where=ticks_num.detach().cpu().numpy()!=0.0)).float().to(self.device)
#         time3 = time.time()
#         print('time diff ticks_num', time3-time2)
#         #------------------------------------------------------------------------------------------
#         #update:
#         self.Q.matrix = (torch.multiply(ticks_num_divided,
#                             f.reduce(lambda x, y: x +  p(y) * T(y),
#                                      range(self.replay_buffer_size))) +  torch.multiply(torch.from_numpy(self.Q.matrix).float().to(self.device),
#                               ( 1.0 - torch.multiply(ticks_num_divided, f.reduce(lambda x, y: x + p(y), range(self.replay_buffer_size) )))  )).detach().cpu().numpy()                                                                                   
#         time4 = time.time()
#         print('time diff ticks_num', time4-time3)
#         self.count_use.matrix = self.count_use.matrix + np.where(ticks_num.detach().cpu().numpy() > 0.0, 1, 0)
        #------------------------------------------------------------------------------------------
        p_1 = f.lru_cache(maxsize=None)(lambda t: np.outer(self.a_f()(self.replay_buffer.action(t)),
                                 self.s_f()(self.replay_buffer.state(t))
                                ))
        
        p_2 = (1.0 + self.s_f.dim * self.a_f.dim)/(self.count_use.matrix + self.s_f.dim * self.a_f.dim)
        
        filter_by_epsilon = f.lru_cache(maxsize=None)(lambda t: np.where(p_1(t) > self.epsilon,
                                                                                          1.0,0.0))
        
        p = f.lru_cache(maxsize=None)(lambda t: np.multiply(filter_by_epsilon(t),
                                  np.multiply(p_1(t), p_2)))
        
        T = f.lru_cache(maxsize=None)(lambda t: self.replay_buffer.reward(t) + self.gamma * np.min(np.matmul(self.Q.matrix,self.s_f()(self.replay_buffer.state(t)))))

        #ticks_num = sum([filter_by_epsilon(t) for t in range(self.replay_buffer_size)])
        time1 = time.time()
        ticks_num = f.reduce(lambda x, y: x + filter_by_epsilon(y), range(self.replay_buffer_size)) 
        time2 = time.time()
        print('time diff ticks_num', time2-time1)
        ticks_num_divided = np.divide(1.0,
                         ticks_num,
                         out = np.zeros_like(ticks_num),
                         where=ticks_num!=0.0)
        time3 = time.time()
        print('time diff ticks_num', time3-time2)
        #------------------------------------------------------------------------------------------
        #update:
        self.Q.matrix = (np.multiply(ticks_num_divided,
                            f.reduce(lambda x, y: x +  p(y) * T(y),
                                     range(self.replay_buffer_size))) +  np.multiply(self.Q.matrix,
                              ( 1.0 - np.multiply(ticks_num_divided, f.reduce(lambda x, y: x + p(y), range(self.replay_buffer_size) )))  ))                                                                                  
        time4 = time.time()
        print('time diff ticks_num', time4-time3)
        self.count_use.matrix = self.count_use.matrix + np.where(ticks_num > 0.0, 1, 0)
        #------------------------------------------------------------------------------------------
        

In [127]:
action_dots = dots(array(
    [[1.0, 1.0]
    , [1.0, 0.0]
    , [1.0, -1.0]
    , [0.0, 1.0]
    , [0.0, 0.0]
    , [0.0, -1.0]
    , [-1.0, 1.0]
    , [-1.0, 0.0]
    , [-1.0, -1.0]
   ],dtype = np.float32))

In [128]:
import itertools

x_partition = [-5.0, -2.0, -1.0 , -0.7, -0.5, -0.3, -0.2, -0.1, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 2.0, 5.0]
y_partition = [-5.0, -2.0, -1.0 , -0.7, -0.5, -0.3, -0.2, -0.1, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 2.0, 5.0]
vx_partition = [-2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0]
vy_partition = [-2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0]

iterables = [x_partition, y_partition, vx_partition, vy_partition]

state_dots = dots(array(
    [t for t in itertools.product(*iterables)],dtype = np.float32))

In [129]:
agent_one = agent(state_dots, action_dots)

In [130]:
agent_one.Q()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [135]:
agent_one.Q_update()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide


time diff ticks_num 79.87548327445984
time diff ticks_num 0.0009980201721191406
time diff ticks_num 81.701815366745


In [131]:
time1 = time.time()
agent_one.s_f()(np.array([0., 0., 0., 0.]))
time2 = time.time()
print('time diff ticks_num', time2-time1)
print(agent_one.s_f()(np.array([0., 0., 0., 0.])))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide


time diff ticks_num 1.593968152999878
[1.7913057e-07 1.9044984e-07 1.9920503e-07 ... 1.9920503e-07 1.9044984e-07
 1.7913057e-07]


In [133]:
aa = agent_one.s_f()(np.array([0., 0., 0., 0.]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
